In [29]:
import pandas as pd
import sqlite3
!pip install pyodbc


In [30]:
csv_file_path = 'Historical_Fire_Data.csv'
df = pd.read_csv(csv_file_path)

df.head()

,RecordID,Incident_ID,AddressRaw,AddressStandardized,AlarmDateTimeAmericaNY,CauseOfIgnition,FireAreaOfOrigin,FireType,HeatSource,IncidentDurationMinutes,...,PropertyUse,StreetName,StreetNumber,StreetPostDirection,StreetPredirection,StreetSuffix,UnitFirstArrive,UnitFirstResponseTimeMinutes,Year,ZipCode
0,1,2000930,2400 FONTAINE AVE,2400 FONTAINE AVE,2002/04/14 16:38:57+00,Undetermined,,Natural vegetation fire,,17.299999,...,Undetermined,FONTAINE,2400,,,AVE,E6,3.50,2002,22901
1,2,2003381,905 FOREST ST,905 FOREST ST,2002/11/07 20:21:02+00,Intentional,Bedroom - < 5 persons; included are jail or pr...,Structure Fire,Cigarette lighter,258.700012,...,1 or 2 family dwelling,FOREST,905,,,ST,C2,3.20,2002,22901
2,3,2003463,219 5TH ST SW,219 5TH ST SW,2002/11/14 12:46:32+00,,,Mobile property (vehicle) fire,,9.400000,...,,5TH,219,SW,,ST,BC2,0.05,2002,22903
3,4,2003769,,,2002/12/11 14:30:15+00,,,Structure Fire,,24.900000,...,1 or 2 family dwelling,,,,,,BC2,1.60,2002,22902
4,5,2003827,822 HARDY DR,822 HARDY DR,2002/12/17 07:38:34+00,Intentional,"Cooking area, kitchen",Structure Fire,"Heat from powered equipment, other",98.800003,...,Multifamily dwelling,HARDY,822,,,DR,E7,5.93,2002,22901


In [33]:
file_path = 'Historical_Fire_Data.csv'
data = pd.read_csv(file_path)

conn = sqlite3.connect('fire_incidents.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS DimDate (
        DateKey INTEGER PRIMARY KEY,
        Date DATE,
        Day INTEGER,
        Month INTEGER,
        Quarter INTEGER,
        Year INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS DimLocation (
        LocationKey INTEGER PRIMARY KEY,
        AddressRaw VARCHAR(255),
        AddressStandardized VARCHAR(255),
        ZipCode INTEGER,
        StreetName VARCHAR(100),
        StreetNumber VARCHAR(20)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS DimCause (
        CauseKey INTEGER PRIMARY KEY,
        CauseOfIgnition VARCHAR(255),
        FireAreaOfOrigin VARCHAR(255),
        FireType VARCHAR(255),
        HeatSource VARCHAR(255)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS FactFireIncidents (
        IncidentID INTEGER PRIMARY KEY,
        DateKey INTEGER,
        LocationKey INTEGER,
        CauseKey INTEGER,
        IncidentDurationMinutes FLOAT,
        FOREIGN KEY (DateKey) REFERENCES DimDate(DateKey),
        FOREIGN KEY (LocationKey) REFERENCES DimLocation(LocationKey),
        FOREIGN KEY (CauseKey) REFERENCES DimCause(CauseKey)
    )
''')
conn.commit()
conn.close()


In [34]:
conn = sqlite3.connect('fire_incidents.db')
cursor = conn.cursor()

data = [
    (1, 2000930, '2400 FONTAINE AVE', '2400 FONTAINE AVE', '2002/04/14 16:38:57+00', 'Undetermined', None, 'Natural vegetation fire', None, 17.299999, None, 'Undetermined', 'FONTAINE', 2400, None, 'AVE', 'E6', 3.50, 2002, 22901),
    (2, 2003381, '905 FOREST ST', '905 FOREST ST', '2002/11/07 20:21:02+00', 'Intentional', 'Bedroom - < 5 persons; included are jail or pr...', 'Structure Fire', 'Cigarette lighter', 258.700012, None, '1 or 2 family dwelling', 'FOREST', 905, None, 'ST', 'C2', 3.20, 2002, 22901),
    (3, 2003463, '219 5TH ST SW', '219 5TH ST SW', '2002/11/14 12:46:32+00', None, None, 'Mobile property (vehicle) fire', None, 9.400000, None, None, '5TH', 219, 'SW', None, 'BC2', 0.05, 2002, 22903),
    (4, 2003769, None, None, '2002/12/11 14:30:15+00', None, None, 'Structure Fire', None, 24.900000, None, '1 or 2 family dwelling', None, None, None, 'BC2', 1.60, 2002, 22902),
    (5, 2003827, '822 HARDY DR', '822 HARDY DR', '2002/12/17 07:38:34+00', 'Intentional', 'Cooking area, kitchen', 'Structure Fire', 'Heat from powered equipment, other', 98.800003, None, 'Multifamily dwelling', 'HARDY', 822, None, 'DR', 'E7', 5.93, 2002, 22901)
]

for row in data:
    if len(row) >= 20:
        # Calculate DateKey
        year = row[19]
        month = int(row[4][5:7]) if row[4][5:7].isdigit() else 1
        day = int(row[4][8:10]) if row[4][8:10].isdigit() else 1
        date_key = year * 10000 + month * 100 + day
        
cursor.execute("SELECT COUNT(*) FROM DimDate WHERE DateKey = ?", (date_key,))
count = cursor.fetchone()[0]
if count == 0:
    cursor.execute('''
        INSERT INTO DimDate (DateKey, Date, Day, Month, Quarter, Year)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (date_key, pd.to_datetime(row[4]).date(), day, month, (month - 1) // 3 + 1, year))

    cursor.execute('''
        INSERT INTO DimLocation (LocationKey, AddressRaw, AddressStandardized, ZipCode, StreetName, StreetNumber)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (row[1], row[2], row[3], row[19], row[12], row[13]))

    cursor.execute('''
        INSERT INTO DimCause (CauseKey, CauseOfIgnition, FireAreaOfOrigin, FireType, HeatSource)
        VALUES (?, ?, ?, ?, ?)
    ''', (row[1], row[5], row[6], row[7], row[8]))

    cursor.execute('''
        INSERT INTO FactFireIncidents (IncidentID, DateKey, LocationKey, CauseKey, IncidentDurationMinutes)
        VALUES (?, ?, ?, ?, ?)
    ''', (row[1], date_key, row[1], row[1], row[9]))

conn.commit()
conn.close()


In [37]:
conn = sqlite3.connect('fire_incidents.db')
cursor = conn.cursor()
cursor.execute('''
    SELECT
        IncidentID,
        AlarmDateTimeAmericaNY,
        AddressRaw,
        CauseOfIgnition,
        IncidentDurationMinutes
    FROM
        FactFireIncidents;
''')
cursor.execute('''
    SELECT
        Year,
        CauseOfIgnition,
        COUNT(IncidentID) AS TotalIncidents
    FROM
        FactFireIncidents
    GROUP BY
        Year, CauseOfIgnition;
''')
conn.commit()

OperationalError: no such column: AlarmDateTimeAmericaNY